## Import libraries

In [25]:
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
import os

In [26]:
cat_directory = './data/lunar/training/catalogs/'
cat_file = cat_directory + 'apollo12_catalog_GradeA_final.csv'
cat = pd.read_csv(cat_file)
cat

## Read the CSV file corresponding to that detection and plot it


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

def list_csv_files(directory):
    try:
        csv_file_names = [os.path.splitext(f)[0] for f in os.listdir(directory) 
                          if f.endswith('.csv') and os.path.isfile(os.path.join(directory, f))]
        return csv_file_names
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

data_directory = './data/lunar/training/data/S12_GradeA/'
files = list_csv_files(data_directory)

for tf in files:
    csv_file = f'{data_directory}{tf}.csv'
    
    # Assuming 'cat' is a DataFrame that you've defined elsewhere
    try:
        index = cat.iloc[cat.index[cat['filename'] == tf].tolist()[0]]
    except IndexError:
        print(f"Filename '{tf}' not found in 'cat' DataFrame.")
        continue  # Skip this iteration if the index is not found

    it = index['time_rel(sec)']
    
    # Read the data
    data_cat = pd.read_csv(csv_file)
    
    # Convert time strings to datetime objects
    csv_times_dt = np.array(data_cat['time_rel(sec)'].tolist())
    
    csv_data = np.array(data_cat['velocity(m/s)'].tolist())
    
    # Create a new figure for each plot
    plt.figure(figsize=(10, 5))  # Adjust figure size as needed
    
    # Plot the trace
    plt.plot(csv_times_dt, csv_data, label='Velocity')
    
    # Make the plot pretty
    plt.xlim((np.min(csv_times_dt), np.max(csv_times_dt)))
    plt.ylabel('Velocity (m/s)')
    plt.title(f'{tf}', fontweight='bold')
    
    # Plot where the arrival time is
    arrival_line = plt.axvline(x=it, c='red', label='Rel. Arrival')
    plt.legend(handles=[arrival_line])
    
    # Show the plot
    plt.xlabel('Time (month-day hour)')
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()  # Display the current plot



## Combining all data into for machine learning prediction

In [ ]:
def list_csv_files(directory):
    try:
        csv_file_names = [os.path.splitext(f)[0] for f in os.listdir(directory) 
                          if f.endswith('.csv') and os.path.isfile(os.path.join(directory, f))]
        return csv_file_names
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

data_directory = './data/lunar/training/data/S12_GradeA/'
files = list_csv_files(data_directory)
target_data = pd.read_csv(cat_file)


# Initialize a DataFrame to hold combined data
combined_data = pd.DataFrame()
# Step 2: Load all seismic data files and merge
for tf in files:
    file = f'{data_directory}{tf}.csv'
    df = pd.read_csv(file)
    
    # Merge target data (this assumes the 'filename' column in target_data)
    target_row = target_data.loc[target_data['filename'] == tf]
# print(len(tr))
    # print(target_row['time_rel(sec)'].values[0])
    # print(df['time_rel(sec)'].values[:])
    if not target_row.empty:
        # Add a new column for the event
        for i in df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'].values[:]:
            df['event'] = 1 if target_row['time_abs(%Y-%m-%dT%H:%M:%S.%f)'][:-10] == i[:-10] else 0
        combined_data = pd.concat([combined_data, df])

# Step 3: Set the index to the time column and sort
combined_data['time'] = pd.to_datetime(combined_data['time_abs(%Y-%m-%dT%H:%M:%S.%f)'])  # Adjust as necessary
combined_data.set_index('time', inplace=True)
combined_data.sort_index(inplace=True)
combined_data

## Data splitting for model training

In [ ]:
X = combined_data.drop(combined_data['event'])
y = combined_data['event']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Using Random Forest 

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
 
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


## Using Gradient Boosting

In [ ]:
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

xgb_classifier.fit(X_train, y_train)

y_pred = xgb_classifier.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


## Using Support Vector Machine

In [ ]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel='linear', random_state=42)

svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))



In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),  # Hidden layer with 16 neurons
    layers.Dense(8, activation='relu'),   # Hidden layer with 8 neurons
    layers.Dense(1, activation='sigmoid') # Output layer
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=5, verbose=1)

# Make predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))
